# Get all sets

In [37]:
import itertools
from collections import namedtuple

In [38]:
# card namedtuple type
Card = namedtuple('card', ['number', 'color', 'shape', 'shade'])

In [39]:
# characteristics
numbers = ['one', 'two', 'three']
colors = ['green', 'purple', 'red']
shapes = ['diamond', 'squiggle', 'oval']
shades = ['open', 'solid', 'striped']

In [40]:
# create all cards 
all_cards = {
    Card(number=number, color=color, shape=shape, shade=shade)
    for number in numbers
    for color in colors
    for shape in shapes
    for shade in shades
}

In [41]:
print(f"There are {len(all_cards)} cards in total.")

There are 81 cards in total.


In [42]:
all_cards_list = list(all_cards)
possible_sets = list(itertools.combinations(all_cards_list, 3))

In [43]:
print(f"There are {len(possible_sets):,} different 3 card combinations in total.")

There are 85,320 different 3 card combinations in total.


In [44]:
def check_if_valid(candidate_set):
    set_numbers = set()
    set_colors = set()
    set_shapes = set()
    set_shades = set()
    
    for card in candidate_set:
        set_numbers.add(card.number)
        set_colors.add(card.color)
        set_shapes.add(card.shape)
        set_shades.add(card.shade)

    set_totals = {len(set_numbers), len(set_colors), len(set_shapes), len(set_shades)}
    
    if set_totals in [{1},{3},{1,3}]:
        return True
    
    return False

In [45]:
valid_sets, invalid_sets = set(), set()

for candidate_set in possible_sets:
    (valid_sets if check_if_valid(candidate_set) else invalid_sets).add(candidate_set)

In [46]:
assert len(valid_sets)+len(invalid_sets) == len(possible_sets)

In [47]:
print(f"There are {len(valid_sets):,} valid sets and {len(invalid_sets):,} invalid sets.")

There are 1,080 valid sets and 84,240 invalid sets.


# Detect cards on a table

In [33]:
import cv2
import numpy as np

In [34]:
image_path = 'test_images/test_2.png'

In [35]:
def detect_and_highlight_cards(image_path):
    image = cv2.imread(image_path)
    orig_image = image.copy()
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0) # GaussianBlur to reduce noise and improve edge detection
    
    edged = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # find contours

    cnt = 0 
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100:  # Minimum area threshold
            continue
        
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            
            # Assuming cards are roughly rectangular with aspect ratio around 1.5 to 2.5
            if 1.3 < aspect_ratio < 2.7:
                cv2.drawContours(orig_image, [approx], -1, (0, 255, 0), 3)
                cnt += 1
                print(cnt)
    
    return orig_image

In [36]:
highlighted_image = detect_and_highlight_cards(image_path)
cv2.imshow('Highlighted Cards', highlighted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9


In [63]:
import os

In [67]:
def count_png_files(root_folder):
    png_count = 0
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.png'):
                png_count += 1
    return png_count

root_folder = 'cards_dataset'  # replace with your root folder path
png_count = count_png_files(root_folder)
print(f"Total number of .png files: {png_count}")

Total number of .png files: 1943


In [65]:
name_mappings = {
    'empty': 'open',
    'full': 'solid',
    'partial': 'striped'
}

base_dir = 'cards_dataset'

def rename_folders(base_dir, name_mappings):
    for root, dirs, files in os.walk(base_dir):
        for dir_name in dirs:
            new_name = name_mappings.get(dir_name)
            if new_name:
                current_path = os.path.join(root, dir_name)
                new_path = os.path.join(root, new_name)
                os.rename(current_path, new_path)
                print(f'Renamed: {current_path} to {new_path}')

rename_folders(base_dir, name_mappings)

Renamed: cards_dataset/one/green/diamond/full to cards_dataset/one/green/diamond/solid
Renamed: cards_dataset/one/green/diamond/empty to cards_dataset/one/green/diamond/open
Renamed: cards_dataset/one/green/diamond/partial to cards_dataset/one/green/diamond/striped
Renamed: cards_dataset/one/green/squiggle/full to cards_dataset/one/green/squiggle/solid
Renamed: cards_dataset/one/green/squiggle/empty to cards_dataset/one/green/squiggle/open
Renamed: cards_dataset/one/green/squiggle/partial to cards_dataset/one/green/squiggle/striped
Renamed: cards_dataset/one/green/oval/full to cards_dataset/one/green/oval/solid
Renamed: cards_dataset/one/green/oval/empty to cards_dataset/one/green/oval/open
Renamed: cards_dataset/one/green/oval/partial to cards_dataset/one/green/oval/striped
Renamed: cards_dataset/one/red/diamond/full to cards_dataset/one/red/diamond/solid
Renamed: cards_dataset/one/red/diamond/empty to cards_dataset/one/red/diamond/open
Renamed: cards_dataset/one/red/diamond/partial t

In [58]:
def rename_files(root_folder):
    count = 1
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.png'):
                old_path = os.path.join(subdir, file)
                new_filename = f"card_{count:05d}.png"
                new_path = os.path.join(subdir, new_filename)
                os.rename(old_path, new_path)
                print(f"Renamed {old_path} to {new_path}")
                count += 1

root_folder = 'cards_dataset'  # replace with your root folder path
rename_files(root_folder)

Renamed cards_dataset/one/green/diamond/solid/card_00005.png to cards_dataset/one/green/diamond/solid/card_00001.png
Renamed cards_dataset/one/green/diamond/solid/card_00011.png to cards_dataset/one/green/diamond/solid/card_00002.png
Renamed cards_dataset/one/green/diamond/solid/card_00010.png to cards_dataset/one/green/diamond/solid/card_00003.png
Renamed cards_dataset/one/green/diamond/solid/card_00004.png to cards_dataset/one/green/diamond/solid/card_00004.png
Renamed cards_dataset/one/green/diamond/solid/card_00012.png to cards_dataset/one/green/diamond/solid/card_00005.png
Renamed cards_dataset/one/green/diamond/solid/card_00006.png to cards_dataset/one/green/diamond/solid/card_00006.png
Renamed cards_dataset/one/green/diamond/solid/card_00007.png to cards_dataset/one/green/diamond/solid/card_00007.png
Renamed cards_dataset/one/green/diamond/solid/card_00013.png to cards_dataset/one/green/diamond/solid/card_00008.png
Renamed cards_dataset/one/green/diamond/solid/card_00017.png to 

In [60]:
def list_png_files(root_folder):
    png_files = []
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.png'):
                png_files.append(file)
    return png_files

root_folder = 'cards_dataset'  # replace with your root folder path
png_files = sorted(list_png_files(root_folder))
print(len(png_files))
for file in png_files:
    print(file)

1527
card_00004.png
card_00005.png
card_00006.png
card_00007.png
card_00010.png
card_00011.png
card_00012.png
card_00013.png
card_00014.png
card_00015.png
card_00016.png
card_00017.png
card_00024.png
card_00026.png
card_00028.png
card_00029.png
card_00030.png
card_00031.png
card_00032.png
card_00033.png
card_00036.png
card_00037.png
card_00038.png
card_00039.png
card_00040.png
card_00041.png
card_00048.png
card_00049.png
card_00050.png
card_00051.png
card_00052.png
card_00053.png
card_00054.png
card_00055.png
card_00056.png
card_00057.png
card_00058.png
card_00059.png
card_00060.png
card_00061.png
card_00062.png
card_00063.png
card_00064.png
card_00065.png
card_00066.png
card_00067.png
card_00070.png
card_00071.png
card_00072.png
card_00073.png
card_00074.png
card_00075.png
card_00076.png
card_00077.png
card_00078.png
card_00079.png
card_00080.png
card_00081.png
card_00082.png
card_00083.png
card_00084.png
card_00085.png
card_00086.png
card_00087.png
card_00088.png
card_00089.png
card_